## 巡回セールスマン問題をQAOAで解く

### 問題設定
- 3つの都市 A, B, C がある
- 移動距離の総和が最小となるような都市の巡回順序 (0,1,2) を考える
- 同じ都市は 2 回訪問しない
- 1 回の訪問につき 1 つの都市を訪問する
- スタートとゴールは同じ都市とする（巡回問題）
- 各都市間の距離は下表により定める（例: 都市 A から都市 B に移動するときの距離は 2 である）
- 以降のコスト関数の定式化では、この表に示された都市間距離 $d_{ij}$ を使用する  
  (例: $d_{0,1}=2$ は都市A→都市Bへの移動距離が2であることを表す)


|       | A  | B | C  |
| ----- | -- | - | -- |
| **A** | 0  | 2 | 9  |
| **B** | 1  | 0 | 6  |
| **C** | 15 | 7 | 0  |

### 二値変数の設定
4都市 $A, B, C$ を順に 0, 1, 2 と番号付けする  
二値変数 $q_{i,j} \in \{0,1\}$ を、
「巡回順序 $i$ に都市 $j$ を訪れる」場合に1をとる変数と定義する

下表は、縦軸を巡回順序 $i$、横軸を都市 $j$ として、二値変数の配置を示す

|       | A      | B      | C      |
| ----- | ------ | ------ | ------ |
| **0** |$q_{0,0}$|$q_{0,1}$|$q_{0,2}$|
| **1** |$q_{1,0}$|$q_{1,1}$|$q_{1,2}$|
| **2** |$q_{2,0}$|$q_{2,1}$|$q_{2,2}$|

### コスト関数
訪問距離 $d$ は、順序 $i$ から次の順序 $(i+1)\;\bmod\;3$ へ移動したときの都市間距離の総和である  
ここで $q_{i,j}\in\{0,1\}$ は「順序 $i$ に都市 $j$ を訪問する」ことを表す二値変数である  

$$
d = \sum_{i=0}^{2}\sum_{j=0}^{2}\sum_{k=0}^{2} d_{j,k} \ast q_{i,j} \ast q_{(i+1)\bmod 3, k}
$$

### コスト関数のハミルトニアン
$q = \frac{1 - z}{2}$の関係により二値変数 $q$ を演算子 $Z$ に置き換えると、コスト関数のハミルトニアン $\hat{H}_d$ は以下のように定式化される

$$
\begin{align*}
\hat{H}_d
  &= \frac{1}{4} \sum_{i=0}^{2}\sum_{j=0}^{2}\sum_{k=0}^{2} d_{j,k} \ast (
  - Z_{i,j} - Z_{(i+1) \bmod 3,k} + Z_{i,j}\;Z_{(i+1) \bmod 3,k}
  ) + const
\end{align*}
$$

### 制約条件

1. **順番(行)方向の制約**（一回の訪問で訪れる都市は1つだけ）：
$$
\sum_{j=0}^{2} q_{i,j} = 1, \quad \forall i = 0,1,2,3
$$

2. **都市方向(列)の制約**（各都市は一度だけ訪問される）：
$$
\sum_{i=0}^{2} q_{i,j} = 1, \quad \forall j = 0,1,2,3
$$

### 制約条件のハミルトニアン
1. **順番(行)方向のハミルトニアン**

$$
\begin{align*}
     \hat{H}_{row} 
     &= \sum_{i=0}^{2}
     \left(
         \sum_{j=0}^{2} q_{i,j} - 1
     \right)^2
\end{align*}
$$

2. **都市(列)方向のハミルトニアン**
$$
\begin{align*}
     \hat{H}_{col} 
     &= \sum_{j=0}^{2}
     \left(
         \sum_{i=0}^{2} q_{i,j} - 1
     \right)^2
\end{align*}
$$

$q = \frac{1 - z}{2}$の関係により二値変数 $q$ を演算子 $Z$ に置き換えると、コスト関数のハミルトニアン $\hat{H}_d$ は以下のように定式化される
$$
\hat{H}_{row} = \frac{1}{2}
    \left (
       - \sum_{i=0}^{2}\sum_{j=0}^{2} Z_{i,j} + \sum_{i=0}^{2}\sum_{j=0}^{2}\sum_{k=j+1}^{2} Z_{i,j}Z_{i,k}
    \right) + const
$$
$$
\hat{H}_{col} = \frac{1}{2}
    \left (
       - \sum_{i=0}^{2}\sum_{j=0}^{2} Z_{i,j} + \sum_{j=0}^{2}\sum_{i=0}^{2}\sum_{k=i+1}^{2} Z_{i,j}Z_{k,j}
    \right) + const
$$


## 時間発展演算子
$A \in \mathbb{R}$を制約条件のハミルトニアンの重み、$\gamma \in \mathbb{R}$を最適化するハイパーパラメータとする    
全体のハミルトニアン $\hat{H}$ を $\hat{H}=\hat{H}_d + A(\hat{H}_{row}+\hat{H}_{col})$により定めると、
時間発展演算子 $U$ は、

\begin{align*}
U &= \exp\!\left( -i\gamma \hat{H} \right) \\
  &= \exp\!\left\lbrack
  -i\gamma \left(
      \hat{H}_d + A\hat{H}_{row}+A\hat{H}_{col}
      \right) 
  \right \rbrack) \\
\end{align*}

となる。トロッター分解の近似を考えればこれは下式のように変形できる。
\begin{align*}
U \rightarrow \exp\!\left\lbrack
  -i\gamma \hat{H}_d
  \right \rbrack
  \exp\!\left\lbrack
  -i\gamma A\hat{H}_{row}
  \right \rbrack
    \exp\!\left\lbrack
  -i\gamma A\hat{H}_{col}
  \right \rbrack
  \\
\end{align*}

### Qiskitによる実装

In [1]:
import numpy as np
from scipy.optimize import minimize
from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.quantum_info import Statevector

In [2]:
# 諸条件

# 量子ビット数
n_qubit = 9

# 繰り返し回数
p = 5

# 距離行列
d = np.array([
    [0,2,9],
    [1,0,6],
    [15,7,0],
])

In [3]:
# 最適化パラメータ
gamma = [Parameter(f"γ{n}") for n in range(p)]
beta = [Parameter(f"β{n}") for n in range(p)]
print(gamma)
print(beta)

[Parameter(γ0), Parameter(γ1), Parameter(γ2), Parameter(γ3), Parameter(γ4)]
[Parameter(β0), Parameter(β1), Parameter(β2), Parameter(β3), Parameter(β4)]


In [4]:
# コスト関数のハミルトニアンの作成

# Z_{i,j}項
def cost_hamiltonian_single_1():
    paulis = []
    coeffs = []

    for i in range(3):
        for j in range(3):
            for k in range(3):
                    q1 = 3*i+j
                    coef = -2*0.25*d[j][k]

                    # Pauli Z のみの文字列を作る
                    Zstring = ['I'] * n_qubit
                    Zstring[q1] = 'Z'

                    paulis.append(''.join(Zstring))
                    coeffs.append(coef)
                
    return SparsePauliOp(paulis, coeffs).simplify().sort()

# Z_{(i+1)%3,k}項
def cost_hamiltonian_single_2():
    paulis = []
    coeffs = []

    for i in range(3):
        for j in range(3):
            for k in range(3):
                    q1 = 3*((i+1)%3)+k
                    coef = -2*0.25*d[j][k]

                    # Pauli Z のみの文字列を作る
                    Zstring = ['I'] * n_qubit
                    Zstring[q1] = 'Z'

                    paulis.append(''.join(Zstring))
                    coeffs.append(coef)
                
    return SparsePauliOp(paulis, coeffs).simplify().sort()

# 相互項
def cost_hamiltonian_pair():
    paulis = []
    coeffs = []

    for i in range(3):
        for j in range(3):
            for k in range(3):
                    q1 = 3*i+j
                    q2 = 3*((i+1)%3)+k 
                    coef = 2*0.25*d[j][k]

                    # Pauli Z のみの文字列を作る
                    Zstring = ['I'] * n_qubit
                    Zstring[q1] = 'Z'
                    Zstring[q2] = 'Z'

                    paulis.append(''.join(Zstring))
                    coeffs.append(coef)
                
    return SparsePauliOp(paulis, coeffs).simplify().sort()

In [5]:
# コスト関数ハミルトニアンの実装
def cost_hamiltonian_gate(gamma):
    
    qc = QuantumCircuit(n_qubit, name="cost_hamiltonian")
    h_1 = cost_hamiltonian_single_1()
    h_2 = cost_hamiltonian_single_2()
    h_p = cost_hamiltonian_pair()

    qc.append(PauliEvolutionGate(operator=h_1, time=gamma), qc.qubits)
    qc.append(PauliEvolutionGate(operator=h_2, time=gamma), qc.qubits)
    qc.append(PauliEvolutionGate(operator=h_p, time=gamma), qc.qubits)
    
    return qc.to_gate()

In [6]:
# 行方向のハミルトニアン

# Z_{i,j}項
def row_hamiltonian_single_1():
    paulis = []
    coeffs = []

    for i in range(3):
        for j in range(3):
            q1 = 3*i+j
            coef = -2*0.5

            # Pauli Z のみの文字列を作る
            Zstring = ['I'] * n_qubit
            Zstring[q1] = 'Z'

            paulis.append(''.join(Zstring))
            coeffs.append(coef)
                
    return SparsePauliOp(paulis, coeffs).simplify().sort()

# 相互項
def row_hamiltonian_pair():
    paulis = []
    coeffs = []

    for i in range(3):
        for j in range(3):
            for k in range(j+1, 3):
                q1 = 3*i+j
                q2 = 3*i+k
                coef = 2*0.5
    
                # Pauli Z のみの文字列を作る
                Zstring = ['I'] * n_qubit
                Zstring[q1] = 'Z'
                Zstring[q2] = 'Z'
    
                paulis.append(''.join(Zstring))
                coeffs.append(coef)
                
    return SparsePauliOp(paulis, coeffs).simplify().sort()

In [7]:
def row_constraint_hamiltonian_gate(gamma, A=20):
    
    qc = QuantumCircuit(n_qubit, name="row_constraint_hamiltonian")
    h_1 = row_hamiltonian_single_1()
    h_p = row_hamiltonian_pair()
    
    qc.append(PauliEvolutionGate(operator=h_1, time=A*gamma), qc.qubits)
    qc.append(PauliEvolutionGate(operator=h_p, time=A*gamma), qc.qubits)
    
    return qc.to_gate()

In [8]:
# 列方向のハミルトニアン

# Z_{i,j}項
def col_hamiltonian_single_1():
    paulis = []
    coeffs = []

    for i in range(3):
        for j in range(3):
            q1 = 3*i+j
            coef = -2*0.5

            # Pauli Z のみの文字列を作る
            Zstring = ['I'] * n_qubit
            Zstring[q1] = 'Z'

            paulis.append(''.join(Zstring))
            coeffs.append(coef)
                
    return SparsePauliOp(paulis, coeffs).simplify().sort()

# 相互項
def col_hamiltonian_pair():
    paulis = []
    coeffs = []

    for j in range(3):
        for i in range(3):
            for k in range(i+1, 3):
                q1 = 3*i+j
                q2 = 3*k+j
                coef = 2*0.5
    
                # Pauli Z のみの文字列を作る
                Zstring = ['I'] * n_qubit
                Zstring[q1] = 'Z'
                Zstring[q2] = 'Z'
    
                paulis.append(''.join(Zstring))
                coeffs.append(coef)
                
    return SparsePauliOp(paulis, coeffs).simplify().sort()

In [9]:
def col_constraint_hamiltonian_gate(gamma, A=20):
    
    qc = QuantumCircuit(n_qubit, name="col_constraint_hamiltonian")

    h_1 = col_hamiltonian_single_1()
    h_p = col_hamiltonian_pair()
    
    qc.append(PauliEvolutionGate(operator=h_1, time=A*gamma), qc.qubits)
    qc.append(PauliEvolutionGate(operator=h_p, time=A*gamma), qc.qubits)

    return qc.to_gate()

In [10]:
def mixer_gate(beta):

    qc = QuantumCircuit(n_qubit, name="mixer")

    for k in range(n_qubit):
        qc.rx(2*beta, k)

    return qc.to_gate()

In [11]:
def create_qaoa_circuit():

    qc = QuantumCircuit(n_qubit, name="QAOA")

    qc.h(qc.qubits)
    
    for i in range(p):
        qc.append(cost_hamiltonian_gate(gamma[i]),qc.qubits)
        qc.append(row_constraint_hamiltonian_gate(gamma[i]),qc.qubits)
        qc.append(col_constraint_hamiltonian_gate(gamma[i]),qc.qubits)
        qc.append(mixer_gate(beta[i]),qc.qubits)
    
    return qc

### 

## 最適化の実行

In [12]:
# コスト関数のハミルトニアン
h_cost = cost_hamiltonian_single_1() + cost_hamiltonian_single_2() + col_hamiltonian_pair()

# 行方向のハミルトニアン
h_row = row_hamiltonian_single_1() + row_hamiltonian_pair()

# 列方向のハミルトニアン
h_col = col_hamiltonian_single_1() + col_hamiltonian_pair()

# 全体のハミルトニアン
h_sum = (h_cost + h_row + h_col).simplify().sort()

In [13]:
# 測定値を得る
def get_expection_value(params):
    
    key = ["β0","β1","β2","β3","β4","γ0","γ1","γ2","γ3","γ4"]
    qc = create_qaoa_circuit()
    dic = dict(zip(key, params))
    
    # 量子回路にパラメータを割り当てる
    assigned_qc = qc.assign_parameters(dic)
    
    # 状態ベクトル
    state = Statevector(assigned_qc)

    # 期待値測定
    exp = state.expectation_value(h_sum)
    
    return float(exp.real)

In [14]:
# 最適解を評価する関数
def get_result(opt_params: list):

    key = ["β0", "β1", "γ0", "γ1"]
    params = dict(zip(key, opt_params))

    opt_qc = create_qaoa_circuit().assign_parameters(params)
    state = Statevector(opt_qc)
    probs = state.probabilities_dict()
    
    best_state = max(probs, key=probs.get)
    best_prob = probs[best_state]

    return (str(best_state), float(best_prob))

In [15]:
# 最適化サイクル

# ループ回数
num_cycles = 20

# 結果保存リスト
history = []

# 初期パラメータ
rng = np.random.default_rng(42)
init_params = rng.random(p*2)

for cycle in range(num_cycles):
    print(f"\n===== Optimization Cycle {cycle + 1} / {num_cycles} =====")

    # 最適化実行
    result = minimize(
        get_expection_value,
        init_params,
        method="COBYLA",
    
    )

    # 結果表示
    best_value = result.fun
    best_params = result.x
    print(f" Best Expectation Value: {best_value}")
    print(f" Best Params: {best_params}")

    # 最適解の評価
    best_state, best_prob = get_result(result.x.tolist())
    print(f" Best State: {best_state}")
    print(f" Best Prob: {best_prob}")

    # 次回の初期値として引き継ぎ
    init_params = best_params.copy()

print("Complete")


===== Optimization Cycle 1 / 20 =====
 Best Expectation Value: -8.244897484207103
 Best Params: [0.77755109 1.4347712  0.86376694 0.69856767 0.09227444 0.97642715
 0.76473188 0.78695185 0.13340781 0.450022  ]
 Best State: 100100100
 Best Prob: 0.044536097410422616

===== Optimization Cycle 2 / 20 =====
 Best Expectation Value: -8.26642714569909
 Best Params: [0.777591   1.43452341 0.86362341 0.69862249 0.09268837 0.97641526
 0.76469917 0.78707147 0.13343073 0.44975638]
 Best State: 100100100
 Best Prob: 0.04432953562097025

===== Optimization Cycle 3 / 20 =====
 Best Expectation Value: -11.797528849555881
 Best Params: [0.77875823 1.43969472 0.86793908 0.68034801 0.12184268 0.97495394
 0.76492287 1.78856919 0.13306967 0.44495265]
 Best State: 110001100
 Best Prob: 0.009523166388686718

===== Optimization Cycle 4 / 20 =====
 Best Expectation Value: -11.975752011723676
 Best Params: [0.77894732 1.43944596 0.86739631 0.67084602 0.13406465 0.97490639
 0.76497973 1.78858876 0.13279123 0.44

### 結果確認

In [16]:
get_result(result.x.tolist())

('000000000', 0.00989905737784044)